In [1]:
!pip install --upgrade langchain langchain-community langgraph openai langchain_openai wikipedia

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=4905b29afe154d86504c2b8378eae526ae9b3c6b9f6a2ddd3f88c2c31cc26321
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
  Attempting uninstall: langchain-core
 

In [4]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=300)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki_tool.run({"query": "AI agents"})

'Page: Intelligent agent\nSummary: In artificial intelligence, an intelligent agent is an entity that perceives its environment, takes actions autonomously to achieve goals, and may improve its performance through machine learning or by acquiring knowledge. Leading AI textbooks define artificial intel'

In [6]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, api_key="sk*******************M7", model="gpt-4o-mini")


In [7]:
tools = [wiki_tool]

# Tool binding
llm_with_tools = llm.bind_tools(tools)

#Tool calling
result = llm_with_tools.invoke("Hello world!")
result
result.content

'Hello! How can I assist you today?'

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

In [9]:
from langchain_core.messages import HumanMessage

#First up, let's see how it responds when there's no need to call a tool:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='6945f09a-4798-49c1-8d4a-e369f165b671'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 83, 'total_tokens': 94, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_72ed7ab54c', 'finish_reason': 'stop', 'logprobs': None}, id='run-15409985-a239-4005-a880-1bbdff55e74a-0', usage_metadata={'input_tokens': 83, 'output_tokens': 11, 'total_tokens': 94, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [10]:
print(response["messages"][-1].content)

Hello! How can I assist you today?


In [11]:
from langchain_core.messages import HumanMessage

#First up, let's see how it responds when there's no need to call a tool:
response = agent_executor.invoke({"messages": [HumanMessage(content="what is agentic ai")]})

response["messages"]

[HumanMessage(content='what is agentic ai', additional_kwargs={}, response_metadata={}, id='cc704242-a6b6-4beb-8540-1189d8ee727d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_nJETD0JhkN3atAANJQ5Jh0zs', 'function': {'arguments': '{"query":"Agentic AI"}', 'name': 'wikipedia'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 86, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-72ada718-03e2-4ff9-ad0b-f1c9013bbc59-0', tool_calls=[{'name': 'wikipedia', 'args': {'query': 'Agentic AI'}, 'id': 'call_nJETD0JhkN3atAANJQ5Jh0zs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'outpu

In [12]:
print(response["messages"][-1].content)

Agentic AI refers to intelligent agents in artificial intelligence that can perceive their environment, take autonomous actions to achieve specific goals, and potentially improve their performance through learning or acquiring knowledge. These agents operate independently and can adapt their behavior based on their experiences.
